In [4]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"ixxxxu","key":"f219641f11918cc26f16646a474eb911"}'}

In [6]:
!ls

kaggle.json  sample_data


In [14]:
# 캐글데이터 -> 코랩 연동

# !mkdir -p ~/.kaggle/ #create folder name Kaggle
# !cp kaggle.json ~/.kaggle/ #copy kaggle.jason into folder Kaggle
# !chmod 600 ~/.kaggle/kaggle.jason

In [15]:
!chmod 600 /content/kaggle.json

- 데이터 출처 -> 캐글 Chest X-Ray Images
- 데이터 소개 -> 중국 광저우 여성 및 어린이 병원의 1-5 세 소아 환자의 흉부 X선 영상
- 데이터 구성 -> 3개의 폴더(train, test, val)로 구성
- 각 이미지 카테고리(폐렴/정상)에 대한 하위 폴더 포함
- 5,856개의 X-Ray 이미지 (JPEG)와 2개의 범주 (폐렴/정상)이 있음
- 병변(폐렴)의 구별 -> 이미지상 다양한 양상의 음영(폐 부위에 희미한 그림자)증가가 관찰

## 0.Set-up
- 패키지 import

In [31]:
import re # 정규표현식 관련된 작업에 필요한 패키지
import os # I/O 관련된 작업에 필요한 패키지

import pandas as pd # 데이터 전처리 관련된 작업에 필요한 패키지
import numpy as np # 데이터 array 작업에 필요한 패키지

import tensorflow as tf # 딥러닝 관련된 작업에 필요한 패키지
import matplotlib.pyplot as plt  # 데이터 시각화에 관련된 작업에 필요한 패키지

from sklearn.model_selection import train_test_split  # 데이터 전처리에 필요한 패키지


## 1.변수 선언

In [32]:
AUTOTUNE = tf.data.experimental.AUTOTUNE # 데이터 로드할 때 빠르게 로드할 수 있도록하는 설정 변수
ROOT_PATH = os.path.join(os.getenv('HOME'), '/content/drive/MyDrive/Kaggle/data') # 데이터 ROOT 경로 변수
BATCH_SIZE = 16 # BATCH_SIZE 변수
IMAGE_SIZE = [180,180] # X-RAY 이미지 사이즈 변수
EPOCHS = 25 # EPOCH 크기 변수

## 2. 데이터 가져오기

In [33]:
train_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/train/*/*'))
test_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/test/*/*'))
val_filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/val/*/*'))

In [34]:
print(len(train_filenames))
print(len(test_filenames))
print(len(val_filenames))

4126
624
0


In [ ]:
# - train 안에는 5216개 , test 안에는 642개, val 안에는 16개의 이미지 존재
# - 비율은 89% , 10,7%, 0.3% 로 validation 이미지가 너무 적어 train에서 일부 가져오기

In [35]:
# train과 val에 있는 데이터를 모은 다음 -> train : val 을 80:20으로 분할
filenames = tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/train/*/*'))
filenames.extend(tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/val/*/*')))

In [36]:
# train, test(val) dataset으로 분할 -> test_size = 0.2 는 20%를 의미
train_filenames, val_filenames = train_test_split(filenames, test_size = 0.2)

print(train_filenames)
print(val_filenames)

['/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person55_bacteria_263.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person1327_bacteria_3374.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person1180_virus_2010.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person772_virus_1401.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person647_virus_1229.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person360_virus_739.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person663_virus_1248.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person516_virus_1033.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person1507_bacteria_3946.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person1280_bacteria_3243.jpeg', '/content/drive/MyDrive/Kaggle/data/chest_xray/train/PNEUMONIA/person348_bac

In [37]:
print(len(train_filenames))
print(len(test_filenames))
print(len(val_filenames))

3357
624
840


In [ ]:
# - train은 3357개 / test는 624 / val은 840개

## 3.데이터 전처리
- train데이터 안에 정상 이미지 수와 폐렴 이미지 수 분포 확인


In [40]:
COUNT_NORMAL = len([filename for filename in train_filenames if "NORMAL" in filename])
print("Normal images count in training set: " + str(COUNT_NORMAL))

COUNT_PNEUMONIA = len([filename for filename in train_filenames if "PNEUMONIA" in filename])
print("Pneumonia images count in training set: " + str(COUNT_PNEUMONIA))

Normal images count in training set: 248
Pneumonia images count in training set: 3109


In [ ]:
# >>>Normal images count in training set : 248
# >>>Pneumonia images count in training set : 3109

- 결과를 보면 정상보다 폐렴 이미지 수가 3배 더 많음
- 사용할 CNN 모델의 경우 데이터가 클래스별 balance가 좋을 수록 training을 잘 함 -> 왜 잘하는지 알아보기
- 실제 임상데이터는 클래스 불균형(Imbalance)인 경우가 많음 -> 이를 어떻게 다룰지에 대한 학습 필요
- val, test 데이터셋은 평가용도 이므로 Imbalance한 데이터 셋이여도 문제가 없음 -> 정말 ?


### tf.data 인스턴스 생성
- tf.data는 tensorflow에서 학습시킬 때, 배치처리 작업을 보다 효율적으로 할 수 있도록 해줌
- 공식 도큐멘트 : Better performance with the ```tf.data API```
- https://www.tensorflow.org/guide/data_performance?hl=ko

In [42]:
train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)
# dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])

### 데이터셋 갯수 확인


In [44]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
print("Training images count: " + str(TRAIN_IMG_COUNT))

VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()
print("Validating images count: " + str(VAL_IMG_COUNT))

Training images count: 3357
Validating images count: 840


In [47]:
print(type(TRAIN_IMG_COUNT))
print(type(VAL_IMG_COUNT))

<class 'numpy.int64'>
<class 'numpy.int64'>


In [29]:
# >>> Training images count: 4185
# >>> Validationg images count: 1047

## 라벨 이름 확인


In [48]:
CLASS_NAMES = np.array([str(tf.strings.split(item, os.path.sep)[-1].numpy())[2:-1]
            for item in tf.io.gfile.glob(str(ROOT_PATH + '/chest_xray/train/*'))])
print(CLASS_NAMES)

['.DS_Store' 'NORMAL' 'PNEUMONIA']


- 두 개의 라벨이 존재
- 정상을 의미하는 'NORMAL'
- 폐렴을 의미하는 'PNEUMONIA'
- 현재 이미지에는 라벨 데이터가 따로 없음 -> 라벨 데이터를 만들어 주는 함수를 생성
- 라벨데이터 = 이미지데이터랑 매칭되는 1이미지 = 0(비정상) 2번 이미지 = 1(정상)

In [49]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep) #폴더 경로를 적용하여 0,1라벨링 -> 폴더계층구조 바뀌면 적용 안될수 있음
    return parts[-2] == "PNEUMONIA" #폐렴이면 양성(True), 정상이면 음성(False)를 리턴

def decode_img(img):
    # 이미지를 unit8 tensor로 변환 -> 왜 인지는 모르겠음 => 연산량 감소효과?
    img = tf.image.decode_jpeg(img, channels = 3)
    # img를 범위 [0,1]의 float32 데이터 타입으로 변환
    img = tf.image.convert_image_dtype(img, tf.float32)
    # img의 이미지 사이즈를 IMAGE_SIZE에서 지정한 사이즈로 수정
    return tf.image.resize(img, IMAGE_SIZE)

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

## train 데이터 셋 & validation 데이터 셋을 만들기
- num_parallel_calls 파라미터에서 set-up에서 초기화 한 AUTOTUNE을 이용하면 더욱 빠르게 데이터를 처리해 줌



In [50]:
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [53]:
# 이미지 리사이즈 결과 확인 및 라벨 맵핑 결과 확인
for image, lable in train_ds.take(1): #take(1)-> 1개의 데이터만 가져온다
    print("Image shape: ", image.numpy().shape)
    print("Label: ", lable.numpy())

test_list_ds = tf.data.Dataset.list_files(str(ROOT_PATH + '/chest_xray/test/*/*'))
TEST_IMAGE_COUNT = tf.data.experimental.cardinality(test_list_ds).numpy()
test_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)

print(TEST_IMAGE_COUNT)

Image shape:  (180, 180, 3)
Label:  True
624


In [ ]:
# >>> 624

## 4.학습 효율화를 위한 함수 정의
- Tensorflow에서는 tf.data -> 파이프라인을 사용하여 학습데이터를 효율적으로 사용할 수 있도록 해 줌

In [54]:
# prepare_for_traininig() 함수는 학습 데이터를 효율적으로 할 수 있도록 데이터를 변환 시켜줌
def prepare_for_traininig(ds, shuffle_buffer_size=1000):

    # shuffle() 을 사용하며 고정 크기 버퍼를 유지하고 해당 버퍼에서 무작위로 균일하게 다음 요소를 선택함 -> 과적합 방지? 배깅?
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # repeat() 을 사용하면 epoch를 진행하면서 여러번 데이터셋을 불러오게 되는데 ->
    # 이때, repeat() 함수를 통해 여러번 데이터셋 활용이 가능
    ds = ds.repeat()

    # batch()를 사용하면 ->  BATCH_SIZE에서 정한 만큼의 배치로 주어짐
    # e.g) 100개의 데이터 -> 10개의 배치로 나누게 되면 각 배치에는 10개의 데이터로 나뉨
    ds = ds.batch(BATCH_SIZE)

    # - prefetch()를 사용하면 학습데이터를 나눠서 읽어 옴
    # - 즉, 첫 번째 데이터를 GPU에서 학습하는 동안 -> 두 번째 데이터를 CPU에서 준비 할 수 있음 (리소스 효율화)
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

train_ds = prepare_for_traininig(train_ds)
val_ds = prepare_for_traininig(val_ds)


## 5.데이터 시각화
- 데이터를 보기 위해 먼저, train에 있는 batch 중 첫 번째 배치를 추출함
- 추출된 배치를 image와 label 데이터 셋으로 나눔
- show_batch() 함수를 이용해서 결과 사진을 시각화


In [56]:
image_batch, label_batch = next(iter(train_ds))

def show_batch(image_batch, label_batch):
    plt.figure(figsize=(10,10))
    for n in range(16):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        if labe_batch[n]:
            plt.title("PNEUMONIA")
        else:
            plt.title("NORMAL")
        plt.axis("off")

show_batch(image_batch.numpy(), label_batch.numpy())


InvalidArgumentError: ignored

## 6.모델링(CNN)을 위한 함수 정의


### 6-1.Convolution block생성
- 1.conv_block()의 구성은 Convolution을 두번 진행
- 2.Batch Normalization을 통해서 기울기 소실( Gradient vanishing, Gradient Exploding)을 해결
- 3.Max Pooling 진행

In [57]:
# 리마인드 : 필터는 이미지의 특징을 찾아내기 위한 공용 파라미터로 일반적으로는 (4,4) 또는 (3,3)의 정사각형의 행렬로 정의됨
# 필터는 지정된 간격 = 순회하는 간격(Stride)으로 순회하며 채널별로 합성곱을 함
# 모든 채널의 합성곱의 합으로 FeatureMap(ActivationMap)을 만듦
def conv_block(filters): #filter <-> kernel = n개의 필터적용시 출력은 n개의 채널을 지니게 됨
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ])
    return block


### 6-2.Dense Block 생성


In [58]:
def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    return block

### 6-3.기존 CNN 모델과 다른점
- Batch Normalization과 Dropout이라는 두가지 regularization(정규화)기법이 동시 사용
- 일반적으로 이런 방법은 잘 사용되지 않거나, 금기시되기도 함 -> 근데 왜씀 ??
- **Understanding the Disharmony between Dropout and Batch Normalization by Variance Shift** 논문에 따르면
- variance shift를 억제하는 Batch Normalization과
- 이를 유발시키는 Dropout을 동시에 사용하는 것이 어울리지 않는다고 밝히고 있음 -> 그러니까 왜씀??
-그러나, 실용적으로는 두 방법을 같이 쓰는 것이 낫다고 보는 견해도 없는 것은 아님
- **Rethinking the Usage of Batch Normalization and Dropout in the Training of Deep Neural Networks** 논문에 따르면
- 예외적으로 동시에 사용하는 것이 성능향상에 도움을 주는 경우를 언급하고 있음 -> 케바케?


## 7.모델링을 통한 regularization(정규화)기법의 적용과 비교


In [59]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),

        tf.keras.layers.Conv2D(16, 3, activation ='relu',padding='same'),
        tf.keras.layers.Conv2D(16, 3, activation ='relu',padding='same'),
        tf.keras.layers.MaxPool2D(),

        conv_block(32),
        conv_block(64),
        conv_block(128),
        tf.keras.layers.Dropout(0.2),
        conv_block(256),
        tf.keras.layers.Dropout(0.2),
       
        tf.keras.layers.Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
    ])
    return model

## 8.데이터 불균형(Imbalance)처리
- 한 라벨이 너무 많은 경우 -> Imbalance 하다고 함
- 해결 방법 -> Weight balancing이라는 테크닉 사용
- training set 의 각 데이터에서 loss 를 계산할 때
- 특정 클래스의 데이터에 더 큰 loss 값을 갖도록 가중치를 부여하는 방법
- Keras에서 model.fit() 을 호출할 때 파라미터로 넘기는 class_weight에 이러한 클래스별 가중치를 세팅할 수 있도록 지원함
- 링크 : https://www.tensorflow.org/tutorials/structured_data/imbalanced_data?hl=ko

### 8-1.딥러닝에서 클래스 불균형을 다루는 방법
- weight_for_0은 'Normal' 이미지에 사용할 weight를,
- weight_for_1은 'Pneumonia' 이미지에 사용할 weight를 세팅
-이 weight들은 'Normal'과 'Pneumonia' 전체 데이터 건수에 반비례하도록 설정됨

In [60]:
weight_for_0 = (1 / COUNT_NORMAL)*(TRAIN_IMG_COUNT)/2.0
weight_for_1 = (1 / COUNT_PNEUMONIA)*(TRAIN_IMG_COUNT)/2.0
class_weight = {0: weight_for_0, 1:weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 6.77
Weight for class 1: 0.54


## 9.모델 훈련
- 이미지 훈련을 위한 GPU사용
- 모델은 위에서 생성한 모델함수인 build_model()을 model에 선언
- 라벨이 2개 밖에 없으므로 "binary_cross entropy" loss를 사용
- optimizer로 'adam'을 사용
- 성과에 대해서 측정하기 위한 metrics으로 'accuracy', 'precision', 'recall'을 사용


In [63]:
with tf.device('/GPU:0'):
    model = build_model()

    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

    model.compile(
        optimize = 'adam',
        loss = 'binary_crossetropy',
        metrics = METRICS,
    )

# 모델 fit -> 각 파라마터에 위에서 선언했었던 변수, 데이터 셋을 가져와서 알맞게 넣어줌
with tf.device('/GPU:0'):
    history = model.fit(
        train_ds,
        steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_ds,
        validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
        class_weight=class_weight
    )

TypeError: ignored

## 10.결과 확인

In [ ]:
fig, ax = plt.subplot(1, 4, figsize=(20,3))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train','val'])

In [ ]:
#테스트 데이터로 모델 평가
#모델 평가를 위해 loss, accuracy, precision, recall 값을 출력
loss, acc, prec, rec = model.evaluate(test_ds)

In [ ]:
# >>> 약 80% 정확도 (CNN)

- 의사들의 평균 판독정확도 90%
- 이를 상회하기 위한 모델 개선 방법 고민 필요
- 하지만 데이터를 구하기 어렵고, 구해도 그 양이 작은 경우가 많음
- 데이터 증강(Data Augmentation)을 사용
- 특히, 의료영상데이터 에서는 인간 장기의 크기가 사람마다 거의 바뀌지 않는 다는 특성을 이용하여
- 데이터 증강 방법으로 노이즈 추가를 주로 사용
- GAN을 이용한 가상환자데이터 생성도 다양한 연구가 진행 중

## 11.폐렴 진단 모델 성능 개선 방향 (단계별)
###  11-1.실험 환경 Set-up
- Batch size, Epoch 등을 변경



### 11-2. 데이터 준비
- 의료영상인 경우, 일반적인 이미지처리에서 사용하는 augmentation들이 항상 도움이 된다고 말할 수 없음
- 특히 X-RAY 같은 의료영상의 특성상, 육안으로도 구분하기 어려운 미묘한 차이에 더해진 노이즈 등 부수효과가
- 오히려 방해를 줄 수도 있다 -> ?? 아까랑 말이 다르네....

In [ ]:
#좌우 반전을 통한 데이터 증강
def augment(image,label):
    image = tf.image.random_flip_left_right(image)  # 랜덤하게 좌우를 반전
    return image,label

def prepare_for_training(ds, shuffle_buffer_size=1000):
    # augment 적용 부분이 배치처리 함수에 추가
    ds = ds.map(augment, num_parallel_calls=2)
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)

In [ ]:
#데이터 증강 적용 후 show_batch() 함수를 통해 실제 적용이 되었는지 여부 확인
#show_batch(image_batch.numpy(), label_batch.numpy())

### 11-3.CNN모델링
- 의료영상 판독을 위해 실습에서 구현했던 model에서 다양한 것들을 바꾸어 가며 실험
- Convolution filter, 채널 개수, activation, 모델구조 등
- 위에서 BatchNormalization과 Dropout을 한 모델 안에서 동시에 사용하는 특이한 구성을 택함


In [ ]:
'''
이것은 일반적으로 잘 사용되는 형태는 아닙니다.
하지만 이미지 사이즈가 크고 데이터가 부족한 의료영상에서는 실용적으로
간혹 좋은 성능을 보이기도 합니다.
만약 이 구성을 변경해 보면 어떤 효과가 발생하는지도 실험해 봅시다.
BatchNormalization을 쓰거나 혹은 쓰지 않거나, Dropout을 쓰거나 혹은 쓰지 않거나 할 수 있습니다.
또, Dropout 비율을 변경해볼 수도 있습니다.
'''

### 11-4. 데이터 불균형 처리


In [ ]:
'''
실습코드에서 데이터의 imbalance 문제에 대처하기 위해
데이터 비율로 나누어진 class_weight를 설정해 주었습니다.
만약 이러한 처리를 생략한다면 어떻게 될까요?
또 recall을 강조하기 위해 폐렴데이터를 잘 맞추는 것을 더 강화하는 효과를 만들어낼 수는 없을까요?
'''

### 11-5. 모델 훈련


In [ ]:
'''
loss 함수를 변경하기는 어렵겠지만,
optimizer나 learning rate 등의 변화를 고려해볼 수 있을 것입니다.
'''

### 11-6.결과 시각화


In [ ]:
'''
테스트데이터로 훈련된 모델을 평가해 봅시다.
우선은 accuracy를 고려해야겠지만
의료영상 모델의 특성상 recall도 중요합니다.
훈련과정의 history 그래프를 시각화해 보고,
학습 진행양상을 면밀히 분석해 보는 것도 잊지 않도록 합시다.
'''